In [1]:
import torch
from transformers import GPT2Model, GPT2Tokenizer
class RewardModel(torch.nn.Module):
    def __init__(self, gpt_model):
        super(RewardModel, self).__init__()
        self.gpt = gpt_model
        self.regression_head = torch.nn.Linear(self.gpt.config.hidden_size, 1)
    def forward(self, input_ids, attention_mask):
        outputs = self.gpt(input_ids=input_ids, attention_mask=attention_mask)
        hidden_states = outputs.last_hidden_state[:, -1, :]  # 取最后一个 Token 的隐藏状态
        reward = self.regression_head(hidden_states)
        return reward

# 加载 GPT 模型和 Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt_model = GPT2Model.from_pretrained("gpt2")
reward_model = RewardModel(gpt_model)

# 训练数据
inputs = tokenizer("This is a sample text.", return_tensors="pt")
labels = torch.tensor([1.0])  # 假设这是高质量文本的奖励

# 训练循环
optimizer = torch.optim.AdamW(reward_model.parameters(), lr=1e-5)
criterion = torch.nn.MSELoss()

for epoch in range(10):
    optimizer.zero_grad()
    rewards = reward_model(inputs["input_ids"], inputs["attention_mask"])
    loss = criterion(rewards, labels)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")


C:\Users\upsun\Desktop\Ai\pytorch\Lib\site-packages\torch\nn\modules\loss.py:538: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1, Loss: 0.7264503240585327
Epoch 2, Loss: 0.20542430877685547
Epoch 3, Loss: 0.04189291223883629
Epoch 4, Loss: 0.0038725496269762516
Epoch 5, Loss: 0.0007453225553035736
Epoch 6, Loss: 0.0064291078597307205
Epoch 7, Loss: 0.009688918478786945
Epoch 8, Loss: 0.008547226898372173
Epoch 9, Loss: 0.005171480122953653
Epoch 10, Loss: 0.0017724260687828064
